In [1]:
import pandas as pd
import httplib2 as http
import json
import requests
import urllib
import bioservices
import mygene
import mechanicalsoup as ms
import csv
from bs4 import BeautifulSoup
import numpy as np

In [2]:
gList=[]
#function to read the genes from the txt file and store them in a gList
with open('9genes_list.txt') as file:
        line = csv.reader(file, delimiter='\n')
        #print(*line)
        genes = list(line)
        for g in genes:
            if str(*g) != '' and not g[0][0].isdigit() :  gList.append(g[0])
            

In [ ]:
gList

In [3]:
len(gList)

23

In [4]:
ak = 'f14aae756e66a7c1a31220c34d6f8c4e'

In [109]:
# FOR TESTING

try:
     from urlparse import urlparse
except ImportError:
     from urllib.parse import urlparse

headers = {
 'Accept': 'application/json',
}

uri = 'https://webservice.thebiogrid.org/interactions/?accesskey=f14aae756e66a7c1a31220c34d6f8c4e&format=json&geneList='

target = uri+'DNASE1L3'

response = requests.get(target)
json_data = json.loads(response.text)

print(json_data)

[]


In [5]:
BioGrid_interactors=[]
pInteractions=[]
def getInteractors(pid):
    #pInteractions=np.DataFrame(columns=('Gene', 'Official_A','Official_B','UniProt_A','UniProt_B','Source'))
    
    uri = 'https://webservice.thebiogrid.org/interactions/?accesskey=f14aae756e66a7c1a31220c34d6f8c4e&format=json&interSpeciesExcluded=true&geneList='
    global pInteractions
    target = uri+pid

    response = requests.get(target)
    json_data = json.loads(response.text)
    #p_interactors = []
    if len(json_data) != 0:
        for k,v in json_data.items():
            tempL=[]
            tempL.append(pid)
            tempL.append(v.get('OFFICIAL_SYMBOL_A'))
            tempL.append(v.get('OFFICIAL_SYMBOL_B'))
            tempL.append(v.get('ENTREZ_GENE_A'))
            tempL.append(v.get('ENTREZ_GENE_B'))
            tempL.append(v.get('SOURCEDB'))
            pInteractions.append(tempL)
            if v.get('OFFICIAL_SYMBOL_A')==pid: g= v.get('OFFICIAL_SYMBOL_B') 
            else: g= v.get('OFFICIAL_SYMBOL_A')
            if g not in BioGrid_interactors: BioGrid_interactors.append(g)
    
        
    return(pInteractions)

In [6]:
pInteractions = pd.read_csv('4.2_interactomeUnion.csv')

In [7]:
BioGrid_interactors = pd.read_csv('BioGrid_interactors.csv')

In [ ]:
BioGrid_interactors

In [218]:
for p in gList:
    getInteractors(p)


In [ ]:
'''
We used ENTREZ ID instead of UniProt id
'''

In [19]:
df = pd.DataFrame(pInteractions, columns=('Gene', 'Official_A','Official_B','UniProt_A','UniProt_B','Source'))

In [20]:
df.head()

,Gene,Official_A,Official_B,UniProt_A,UniProt_B,Source
0,BANK1,BANK1,LYN,55024,4067,BIOGRID
1,BANK1,BANK1,ITPR1,55024,3708,BIOGRID
2,BANK1,BANK1,LYN,55024,4067,BIOGRID
3,BANK1,YWHAQ,BANK1,10971,55024,BIOGRID
4,C4A,C4A,CST3,720,1471,BIOGRID


In [28]:
df.describe()

,Gene,Official_A,Official_B,UniProt_A,UniProt_B,Source
count,1159,1159,1159,1159,1159,1159
unique,21,317,419,319,421,2
top,RIPK1,TNFAIP3,RIPK1,7128,8737,BIOGRID
freq,333,136,234,136,234,1153


In [255]:
# create list of protien interacting with at least one seed gene:
# same BioGrid_interactors
'''
nonSeedGenes =[]
for index, row in df.iterrows():
    if row['Official_A'] == row['Gene']: 
        if row['Official_B'] not in nonSeedGenes and row['Official_B'] not in gList: 
            nonSeedGenes.append(row['Official_B'])
    else : 
        if row['Official_A'] not in nonSeedGenes and row['Official_A'] not in gList: 
            nonSeedGenes.append(row['Official_A'])
'''

BioGrid_interactors_notSeedGene = list(set(BioGrid_interactors) - set(gList))

In [257]:
len(nonSeedGenes)

NameError: name 'nonSeedGenes' is not defined

In [256]:
len(BioGrid_interactors_notSeedGene)

547

In [207]:
pInteractionsAll = []
def getInteractorsAll(pid):
    #pInteractions=np.DataFrame(columns=('Gene', 'Official_A','Official_B','UniProt_A','UniProt_B','Source'))
    
    uri = 'https://webservice.thebiogrid.org/interactions/?accesskey=f14aae756e66a7c1a31220c34d6f8c4e&format=json&interSpeciesExcluded=true&geneList='
    global pInteractionsAll
    target = uri+pid

    response = requests.get(target)
    json_data = json.loads(response.text)
    #p_interactors = []
    if len(json_data) != 0:
        for k,v in json_data.items():
            tempL=[]
            tempL.append(pid)
            tempL.append(v.get('OFFICIAL_SYMBOL_A'))
            tempL.append(v.get('OFFICIAL_SYMBOL_B'))
            tempL.append(v.get('ENTREZ_GENE_A'))
            tempL.append(v.get('ENTREZ_GENE_B'))
            tempL.append(v.get('SOURCEDB'))
            #pInteractionsAll.append(tempL)
            if v.get('OFFICIAL_SYMBOL_A')==pid: g= v.get('OFFICIAL_SYMBOL_B') 
            else: g= v.get('OFFICIAL_SYMBOL_A')
            if g in BioGrid_interactors_notSeedGene: pInteractionsAll.append(tempL)
            
        
    return(pInteractionsAll)

In [52]:
int_dict_all = {}
for g in BioGrid_interactors_notSeedGene:
    int_dict_all[str(g)] = getInteractorsAll(g)


In [258]:
len(pInteractionsAll)

0

In [55]:
df_all = pd.DataFrame(pInteractionsAll, columns=('Gene', 'Official_A','Official_B','UniProt_A','UniProt_B','Source'))

In [56]:
df.to_csv('df.csv', header=True)
df_all.to_csv('df_all.csv', header=True)


In [63]:
#import numpy as np
#np.savetxt("seedGenes.csv", gList, delimiter=",", fmt='%s', header="Seed Gene")
np.savetxt("BioGrid_interactors.csv", BioGrid_interactors, delimiter=",", fmt='%s', header="BioGrid Interactors Gene")
np.savetxt("BioGrid_interactors_notSeedGene.csv", BioGrid_interactors_notSeedGene, delimiter=",", fmt='%s', header="BioGrid Interactors Gene Not Seed Gene")

In [259]:
sg_interactom_bg = pd.read_csv('4.1_BioG_interactomeSeedGene.csv')

In [262]:
for i, row in sg_interactom_bg.iterrows():
    if row[2] not in gList and row[3] not in gList:
        sg_interactom_bg = sg_interactom_bg.drop(i)
        print("999")

In [263]:
sg_union_interactom_bg = pd.read_csv('df.csv')

In [264]:
sg_union_interactom_bg.head()

,Unnamed: 0,Gene,Official_A,Official_B,UniProt_A,UniProt_B,Source
0,0,BANK1,BANK1,LYN,55024,4067,BIOGRID
1,1,BANK1,BANK1,ITPR1,55024,3708,BIOGRID
2,2,BANK1,BANK1,LYN,55024,4067,BIOGRID
3,3,BANK1,YWHAQ,BANK1,10971,55024,BIOGRID
4,4,C4A,C4A,CST3,720,1471,BIOGRID


In [ ]:
for i, row in sg_union_interactom_bg.iterrows():
    if row[2] not in gList and row[3] not in gList:
        sg_union_interactom_bg = sg_union_interactom_bg.drop(i)
        print("999")

In [240]:
IID_interactors=[]
pInteractionsIID = pd.DataFrame( columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])

def getInteractorsIID(query, specie = "human"):
    global IID_interactors, pInteractionsIID, gList
    browser = ms.StatefulBrowser()
    url = "http://178.128.224.72"
    browser.open(url)
    #browser.get_current_page().find_all('textarea')

    sf=browser.select_form()
    
    sf.set_textarea({"ids_area":query})   # query is the line
    
    #print(query)
    name_select = {"name":"in_species"}
    sf.set_select({"in_species":specie})
    # print(browser.get_current_page())
    # sf.print_summary()
    sf.choose_submit("submit")
    name_select = {"name": "submit", "value":"View Results"}
    browser.submit_selected()
    Results = browser.get_current_page()
    ppi_interactions=[]
    
    scrs = Results.findAll('script')
    #resTxt = scrs[scrsText()[:5] == 'res =']
    resTxt = (scrs[::-1][4])
    #print(resTxt)
    y1 = str(resTxt).split('res =')[1].split('; res_headings')[0][3:-2].replace('\"','').replace('\'','')
    y2 = y1.split('], [')
    #print(y2)
    tstdf = pd.DataFrame( columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])
    #tst1 = tst.split('res =')[1].split('; res_headings')[0][3:-2].replace('\'','').replace('"','')
    #tst2 = tst1.split('], [')
    #print(tst1)
    for l in y2:
            tm = l.split(',')
            if str(tm[2].split(';')[0]).strip() not in gList and str(tm[3].split(';')[0]).strip() not in gList: print(str(tm[2])+' '+str(tm[3]))
            else:
                    if str(tm[2].split(';')[0]).strip() in gList: g= str(tm[3].split(';')[0]).strip() 
                    if str(tm[3].split(';')[0]).strip() in gList: g= str(tm[2].split(';')[0]).strip()
                    if g not in IID_interactors: IID_interactors.append(g)

                    tstdf.loc[len(tstdf)] = [*tm]
    pInteractionsIID.append(tstdf)
#    browser.close()
    return tstdf 



In [92]:
getInteractorsIID(gList[1])

,Uni1,Uni2,Sym1,Sym2,evi
0,P07948,Q8NDB2,LYN,BANK1,exp
1,P27348,Q8NDB2,YWHAQ,BANK1,exp
2,Q14643,Q8NDB2,ITPR1,BANK1,exp;pred
3,Q13077,Q8NDB2,TRAF1,BANK1,ortho
4,Q14571,Q8NDB2,ITPR2,BANK1,exp;pred
5,P40763,Q8NDB2,STAT3,BANK1,ortho;pred
6,P16157,Q8NDB2,ANK1,BANK1,pred
7,P63104,Q8NDB2,YWHAZ,BANK1,exp


In [ ]:
for g in gList:
    print(g)
    print(getInteractorsIID(g))
pInteractionsIID

In [241]:
import os
#getInteractorsIID('\n '.join(gList))
#str("\\n ".join(gList))
ttt =''''''
for g in gList:
    ttt +=g+os.linesep
#ttt = str(g+"\n" for g in gList)
ttt



'BANK1\nC4A\nC4B\nC4B_2\nCR2\nCRP\nCTLA4\nDNASE1\nDNASE1L3\nFCGR2B\nIRF5\nITGAM\nLTK\nNCF2\nPDCD1\nPTPN22\nRASGRP1\nRIPK1\nSTAT4\nTLR5\nTNFAIP3\nTNFSF4\nTREX1\n'

In [242]:
pInteractionsIID = getInteractorsIID(ttt)

In [243]:
len(IID_interactors)

2205

In [244]:
#pInteractionsIID
len(pInteractionsIID)

3668

In [245]:
tttt =''''''
for g in IID_interactors:
    tttt +=g+os.linesep

In [ ]:
tttt

In [247]:
IID_interactorsAll=[]
pInteractionsIIDAll = pd.DataFrame( columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])

def getInteractorsIIDAll(query, specie = "human"):
    global IID_interactorsAll, pInteractionsIIDAll, gList
    browser = ms.StatefulBrowser()
    url = "http://178.128.224.72"
    browser.open(url)
    #browser.get_current_page().find_all('textarea')

    sf=browser.select_form()
    
    sf.set_textarea({"ids_area":query})   # query is the line
    
    #print(query)
    name_select = {"name":"in_species"}
    sf.set_select({"in_species":specie})
    # print(browser.get_current_page())
    # sf.print_summary()
    sf.choose_submit("submit")
    name_select = {"name": "submit", "value":"View Results"}
    browser.submit_selected()
    Results = browser.get_current_page()
    ppi_interactions=[]
    
    scrs = Results.findAll('script')
    #resTxt = scrs[scrsText()[:5] == 'res =']
    resTxt = (scrs[::-1][4])
    #print(resTxt)
    y1 = str(resTxt).split('res =')[1].split('; res_headings')[0][3:-2].replace('\"','').replace('\'','')
    y2 = y1.split('], [')
    #print(y2)
    tstdf = pd.DataFrame( columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])
    #tst1 = tst.split('res =')[1].split('; res_headings')[0][3:-2].replace('\'','').replace('"','')
    #tst2 = tst1.split('], [')
    #print(tst1)
    for l in y2:
            tm = l.split(',')
            if str(tm[2].split(';')[0]).strip() not in IID_interactorsAll and str(tm[3].split(';')[0]).strip() not in IID_interactorsAll: print(str(tm[2])+' '+str(tm[3]))
            else:
                    if str(tm[2].split(';')[0]).strip() in IID_interactorsAll: g= str(tm[3].split(';')[0]).strip() 
                    if str(tm[3].split(';')[0]).strip() in IID_interactorsAll: g= str(tm[2].split(';')[0]).strip()
                    if g not in IID_interactorsAll: IID_interactorsAll.append(g)

                    tstdf.loc[len(tstdf)] = [*tm]
    pInteractionsIIDAll.append(tstdf)
#    browser.close()
    return tstdf

In [ ]:
pInteractionsIIDAll = getInteractorsIIDAll(tttt)

In [250]:
len(IID_interactorsAll)

0

In [35]:
tst = '''<script>
                res = [["P07948", "Q8NDB2", "LYN", "BANK1", "exp"], ["P27348", "Q8NDB2", "YWHAQ", "BANK1", "exp"], ["Q14643", "Q8NDB2", "ITPR1", "BANK1", "exp;pred"], ["Q13077", "Q8NDB2", "TRAF1", "BANK1", "ortho"], ["Q14571", "Q8NDB2", "ITPR2", "BANK1", "exp;pred"], ["P40763", "Q8NDB2", "STAT3", "BANK1", "ortho;pred"], ["P16157", "Q8NDB2", "ANK1", "BANK1", "pred"], ["P63104", "Q8NDB2", "YWHAZ", "BANK1", "exp"]]; res_headings = ['UniProt1', 'UniProt2', 'symbol1', 'symbol2', 'evidence type'];
            </script>'''

In [87]:

tst1 = tst.split('res =')[1].split('; res_headings')[0][3:-2].replace('\'','').replace('"','')
tst2 = tst1.split('], [')
#print(tst2)
tstdf = pd.DataFrame( columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])
#tstdf = pd.DataFrame(np.array(tst2).reshape(len(tst2)%5 + 1, 5), columns = ['Uni1', 'Uni2', 'Sym1', 'Sym2', 'evi'])
for l in tst2:
    tm = l.split(',')[:5]
    #print(tm)
    tstdf.loc[len(tstdf)] = [*tm]

tstdf

,Uni1,Uni2,Sym1,Sym2,evi
0,P07948,Q8NDB2,LYN,BANK1,exp
1,P27348,Q8NDB2,YWHAQ,BANK1,exp
2,Q14643,Q8NDB2,ITPR1,BANK1,exp;pred
3,Q13077,Q8NDB2,TRAF1,BANK1,ortho
4,Q14571,Q8NDB2,ITPR2,BANK1,exp;pred
5,P40763,Q8NDB2,STAT3,BANK1,ortho;pred
6,P16157,Q8NDB2,ANK1,BANK1,pred
7,P63104,Q8NDB2,YWHAZ,BANK1,exp


#### summary

---| BioGrid | IID
---|---|---
No. Seed Gene | 23 | 23
No. Interacting Protien | 563 | 2236
No. Interactions | 28294 | 632850

In [82]:
# 4.1 seed gene interactome
cols =['id','A Symbol','B Symbol', 'A UniProt', 'B UniProt', 'DB Source']
interactomeSeedGene = pd.DataFrame(columns=cols)
for i in pInteractions:
    if i[1] in gList and i[2] in gList:
        interactomeSeedGene = interactomeSeedGene.append(pd.Series(i, index=cols), ignore_index=True)
interactomeSeedGene

,id,A Symbol,B Symbol,A UniProt,B UniProt,DB Source
0,RIPK1,RIPK1,TNFAIP3,8737,7128,BIOGRID
1,RIPK1,TNFAIP3,RIPK1,7128,8737,BIOGRID
2,RIPK1,TNFAIP3,RIPK1,7128,8737,BIOGRID
3,RIPK1,TNFAIP3,RIPK1,7128,8737,BIOGRID
4,RIPK1,RIPK1,TNFAIP3,8737,7128,BIOGRID
5,RIPK1,TNFAIP3,RIPK1,7128,8737,BIOGRID
6,TNFAIP3,RIPK1,TNFAIP3,8737,7128,BIOGRID
7,TNFAIP3,TNFAIP3,RIPK1,7128,8737,BIOGRID
8,TNFAIP3,TNFAIP3,RIPK1,7128,8737,BIOGRID
9,TNFAIP3,TNFAIP3,RIPK1,7128,8737,BIOGRID


In [83]:
#interactomeSeedGene.to_csv('4.1_interactomeSeedGene.csv', header=True)

In [85]:
# 4.2 Union interactome
cols =['id','A Symbol','B Symbol', 'A UniProt', 'B UniProt', 'DB Source']
interactomeUnion = pd.DataFrame(columns=cols)
for i in pInteractionsAll:
    if i[1] in gList or i[2] in gList:
        interactomeUnion = interactomeUnion.append(pd.Series(i, index=cols), ignore_index=True)
interactomeUnion

,id,A Symbol,B Symbol,A UniProt,B UniProt,DB Source
0,Tab2,TAB2,RIPK1,23118,8737,BIOGRID
1,Tab2,TAB2,RIPK1,23118,8737,BIOGRID
2,Ctla4,AP2M1,CTLA4,1173,1493,BIOGRID
3,Ctla4,CD80,CTLA4,941,1493,BIOGRID
4,Ctla4,ELAVL1,CTLA4,1994,1493,BIOGRID
5,Ctla4,CD80,CTLA4,941,1493,BIOGRID
6,Ctla4,CD86,CTLA4,942,1493,BIOGRID
7,Ctla4,FYN,CTLA4,2534,1493,BIOGRID
8,Ctla4,STAT5A,CTLA4,6776,1493,BIOGRID
9,Ctla4,STAT5B,CTLA4,6777,1493,BIOGRID


In [55]:
#interactomeUnion.to_csv('4.2_interactomeUnion.csv', header=True)
interactomeUnion = pd.read_csv('4.2_interactomeUnion.csv')
interactomeUnion = interactomeUnion.drop('Unnamed: 0', axis = 1)

In [18]:
maplist = pd.read_csv('genezunip.txt', sep='\t', header=None)

In [56]:
interactomeUnion.head()

,id,A Symbol,B Symbol,A UniProt,B UniProt,DB Source
0,Tab2,TAB2,RIPK1,23118,8737,BIOGRID
1,Tab2,TAB2,RIPK1,23118,8737,BIOGRID
2,Ctla4,AP2M1,CTLA4,1173,1493,BIOGRID
3,Ctla4,CD80,CTLA4,941,1493,BIOGRID
4,Ctla4,ELAVL1,CTLA4,1994,1493,BIOGRID


In [ ]:
#maplist = maplist.drop(2, axis = 1)
mapID = {}
for i, row in maplist.iterrows():
    mapID[row[0]] = row[1]
mapID

In [61]:
#interactomeUnion = interactomeUnion.replace(mapID)
interactomeUnion.to_csv('interactomeUnion_uniprot.csv')

In [ ]:
for i,row in interactomeUnion.iterrows():
    print(row[3])

In [9]:
BioG_interactomeSeedGene = pd.read_csv('4.1_BioG_interactomeSeedGene.csv')

#BioG_interactomeSeedGene = BioG_interactomeSeedGene.replace(mapID)
BioG_interactomeSeedGene.to_csv('4.1_BioG_interactomeSeedGene_uniprot.csv')

In [76]:
# 4.3 Intersection interactome

# TO-DO after getting Ale data
iid_interactomUnion.head()

,UniProt1,UniProt2,symbol1,symbol2,evidence type
0,A0PJW6,Q15116,TMEM223,PDCD1,exp
1,A1KZ92,P19878,PXDNL,NCF2,pred
2,A1X283,P19878,SH3PXD2B,NCF2,pred
3,A2BHY4,P0C0L4,-,C4A,pred
4,A2BHY4,P0C0L5,-,C4B;C4B_2,pred


In [10]:
interactomeUnion.head()
#iid_interactomUnion = pd.read_csv('4.2_union_iid_interactom.csv')
#iid_interactomUnion = iid_interactomUnion.drop('Unnamed: 0', axis=1)
#iid_interactomUnion.head()
#interactomeUnion.columns=['id','symbol1','symbol2', 'UniProt1', 'UniProt2', 'DB Source']

In [79]:
intersect_1 = pd.merge(iid_interactomUnion, interactomeUnion, how='inner', on=['symbol1', 'symbol2'])
interactomeUnion.columns=['id','symbol2','symbol1', 'UniProt2', 'UniProt1', 'DB Source']
intersect_2 = pd.merge(iid_interactomUnion, interactomeUnion, how='inner', on=['symbol1', 'symbol2'])



In [82]:
#intersect = pd.concat([intersect_1,intersect_2])
intersect = intersect.drop(['DB Source','UniProt1_y','UniProt2_y','evidence type','id'], axis=1)
intersect

,UniProt1_x,UniProt2_x,symbol1,symbol2
0,O14763,P21580,TNFRSF10B,TNFAIP3
1,O14763,Q13546,TNFRSF10B,RIPK1
2,O14763,Q13546,TNFRSF10B,RIPK1
3,O14896,Q13568,IRF6,IRF5
4,O14920,P21580,IKBKB,TNFAIP3
5,O14920,P21580,IKBKB,TNFAIP3
6,O14980,Q13546,XPO1,RIPK1
7,O14980,Q13546,XPO1,RIPK1
8,O15111,Q13546,CHUK,RIPK1
9,O15111,Q13568,CHUK,IRF5


In [83]:
intersect.to_csv('intersectecions.csv')

In [84]:
type(pInteractionsAll)

list

In [212]:
#print(len(int_dict))
#for key in int_dict.items():
#    print(len(key[1]))
print(sum(len(k[1]) for k in int_dict.items()))    

610


In [213]:
print(len(BioGrid_interactors))

543


In [ ]:
for g in gList:
    print(g)
    print(getInteractors(g))

In [ ]:
print(*BioGrid_interactors)

In [ ]:
def getUniProt_OS(of):
    

In [97]:
#gList[0]
interactors = []
for k,v in json_data.items():
    #print(k)
    #print(v.get('OFFICIAL_SYMBOL_B'))
    symb_b = v.get('OFFICIAL_SYMBOL_B')
    symb_a = v.get('OFFICIAL_SYMBOL_A')
    
    
    if g not in interactors: interactors.append(g)
    #print(v.get('OFFICIAL_SYMBOL_B'))
    #break
print(*interactors)

LYN ITPR1 BANK1


In [186]:
type(int_dict)

dict

In [262]:
def print_intrs(p):
    output='''<div id="container" style="display: flex;flex-flow: row wrap;justify-content: space-around;">
    <div width="20%" style="min-width:200px; border:1px solid gray; text-align:center; float: left;display: table-cell; background-color: #eee; margin-bottom: 1em;margin-left: 3%;">
            <table>
                <tr>
                    <th>'''+str(p)+''' interactions</th>
                </tr>'''
    for i in getInteractors(p):
        output+='''<tr>
                        <td>'''+str(i)+'''</td>
                    </tr>'''
    output+='</table></div>'
    return(display(HTML(output)))


# ONLYTHIS ONE
def print_bg_intrs_(int_dict):
    output='''<div id="divContainer" width="100%" align="left"
                        style="content-align:left; border:1px solid gray; background-color: #eee;">'''
    for k,v in int_dict.items():
        output+='''<div style="float:left;background-color: #fff;">
                        <div width="100%"><b>'''+str(k)+''' interactions</b></div>
                        <div style="float:left;background-color: #fff;">
                '''
        for i in v:
            output+='''<div style="float:left; margin:2px;padding:1px; border:1px solid #ddd">'''+str(i)+'''</div>'''
        output+='</div></div>'
    output+='</div>'
    return(display(HTML(output)))

def print_bg_intrs(int_dict):
    output='''<div id="divContainer" width="100%" align="left"
                        style="content-align:left; border:1px solid gray; background-color: #eee;">'''
    for k,v in int_dict.items():
        output+=''' <b>'''+str(k)+''' interactions</b></br>
                '''
        for i in v:
            output+='''<div style="float:left; margin:2px;padding:1px; border:1px solid #ddd">'''+str(i)+'''</div>'''
        output+='</br>'
    output+='</div>'
    return(display(HTML(output)))

In [263]:
from IPython.display import HTML, display

print_bg_intrs(int_dict)

In [ ]:
def 

In [ ]:
# Alessandara

In [266]:
def web_scrap_list(query):
    browser = mechanicalsoup.StatefulBrowser()
    url = "http://iid.ophid.utoronto.ca/iid/Search_By_Proteins/"
    browser.open(url)
    #print(browser.get_current_page())
    browser.get_current_page().find_all('textarea')
    sf=browser.select_form()
    #sf.print_summary()
    sf.set_textarea({"ids_area":query})   # query is the line
    browser.submit_selected()
    # sf["includeSource"] = " Source information: detection methods, PubMed IDs, reporting databases"
    Results = browser.get_current_page()
    ppi_interactions=[]
    
    try:
        results_findall = Results.findAll('table')[0].findAll('tr')[1:]
    except Exception:
        return ppi_interactions
        
    count=0
    for row in results_findall:
        #print(block, type(block))
        cells = row.find_all("td")
        print(cells)
        print(cells[1])
        queryUP = cells[1].split("<td>")
        interUP = cells[2]
        querySym = cells[3]
        interSym = cells[4]
        ppi_interactions.append([querySym, interSym, queryUP, interUP])
    return ppi_interactions

In [ ]:
# dictionary with keys Gene and values list of all interactions
genes_interactions = {}  
lst = gList[:3]
for el in lst:
    interactions = web_scrap_list(el) 
    genes_interactions[el] = interactions

In [2]:
import mygene
mg = mygene.MyGeneInfo()

In [ ]:
mg.query('BANK1')